In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np
import os
import warnings
from tqdm import tqdm
import re
import dask.dataframe as dd
warnings.filterwarnings('ignore')

# 1. 전처리 완료 신청현황 데이터 셋 가져오기

In [2]:
apply=pd.read_csv(os.listdir()[2],encoding='EUC-KR')
apply=apply.query('비고!=2021') #2021년도 데이터 셋 삭제
selected_columns=['비고', '지자체명_시도', '지자체명_시군구', '구분', '주소지 (조합, 법인 소재지)', '농업경영체', '배정신청 인원','지자체추가배정인원','합계', '작물 종류'] #필요한 컬럼만 추출
apply=apply[selected_columns]
apply['작물 종류']=apply['작물 종류'].astype(str)
apply['주소지 (조합, 법인 소재지)']=apply['주소지 (조합, 법인 소재지)'].astype(str)

# 2. 농업경영체 등록정보 데이터 필터링 및 중복 행 제거

In [3]:
Regit_Inf=pd.read_excel(os.listdir()[12], usecols=['농업경영체','번호','농지면적(실제경작)','재배품목','시설종류'])
Regit_Inf=Regit_Inf[Regit_Inf['번호'].str.contains('등록')]
duplicated_row=Regit_Inf.duplicated() #중복행 뽑아보기
Regit_final=Regit_Inf[-duplicated_row] #중복행 제거

# 3. 허용작물 분류체계데이터 셋 불러오기

In [4]:
category=pd.read_excel(os.listdir()[5],usecols=['허용작물(법무부)','소분류명(농림부)'])
category=category.dropna(axis=0)

# 4. 농업등록 경영체 작물종류 컬럼 결합하기

## 4.1 노지재배 데이터 셋 구성하기

In [5]:
Regit_노지=Regit_final[Regit_final['시설종류'].isnull()]

In [6]:
duplicated_row=category['소분류명(농림부)'].duplicated(keep='last') #첫번째로 겹치는 행을 반환
category_노지=category[-duplicated_row]
Regit_노지_1=pd.merge(Regit_노지,category_노지, left_on='재배품목', right_on='소분류명(농림부)',how='left')

## 4.3 시설재배 등록정보 데이터 셋

In [7]:
Regit_시설=Regit_final[Regit_final['시설종류'].notnull()]
duplicated_row=category['소분류명(농림부)'].duplicated(keep='first')
category_시설=category[-duplicated_row]
Regit_시설_1=pd.merge(Regit_시설,category_시설, left_on='재배품목', right_on='소분류명(농림부)',how='left')

# 5.3 노지, 시설 bind 해주기

In [8]:
Regit_final=pd.concat([Regit_시설_1,Regit_노지_1],ignore_index=True)
Regit_final=Regit_final[Regit_final['허용작물(법무부)'].notnull()]
Regit_final.drop(columns=['소분류명(농림부)'],inplace=True)

# 6. 재배면적 추출

In [9]:
new_column={'허용작물(법무부)':'작물 종류'}
Regit_final.rename(columns=new_column,inplace=True)
Regit_final_1=Regit_final[['농업경영체','농지면적(실제경작)','작물 종류']]
test_data=pd.merge(apply, Regit_final_1, on=['농업경영체','작물 종류'] ,how='left')

## 6.1 모든 컬럼 값 기준 그룹화-> 농지면적이 가장 큰 행들만 뽑아서 데이터프레임

In [10]:
group_var=['비고', '지자체명_시도', '지자체명_시군구', '구분', '주소지 (조합, 법인 소재지)', '농업경영체', '합계','작물 종류']

In [11]:
최대농지면적=test_data.groupby(group_var).max('농지면적(실제경작)').reset_index()
최대농지면적=최대농지면적.sort_values('농업경영체')

# 6.2 농지면적 0인 행 골라내서 정제하기

In [12]:
not_match=최대농지면적[최대농지면적['농지면적(실제경작)'].isnull()]
match_data=최대농지면적[최대농지면적['농지면적(실제경작)'].notnull()]
not_match.drop(columns=['농지면적(실제경작)'],inplace=True) #농지면적 삭제

# Not match data 재배면적 및 작뭉 종류 nan 값 추가하기

In [13]:
not_match1=pd.merge(not_match, Regit_final_1, on=['농업경영체'] ,how='left') #실제 진행용 데이터

In [14]:
group_var=['비고', '지자체명_시도', '지자체명_시군구', '구분', '주소지 (조합, 법인 소재지)', '농업경영체', '합계','작물 종류_x']
not_match_final=not_match1.groupby(group_var).max('농지면적(실제경작)').reset_index()
not_match_final=not_match_final[not_match_final['농지면적(실제경작)'].notnull()]
not_match_final=not_match_final.sort_values('농업경영체')
#중복제거
duplicated_row=not_match1.duplicated() #중복행 뽑아보기
not_match1=not_match1[-duplicated_row] #중복행 제거

# 병렬처리

https://github.com/eumtaewon/multiprocessing-

In [15]:
not_match1_idx=not_match1.index
not_match_final_idx=not_match_final.index
실제_작물종류_list=[]


for i in tqdm(not_match_final_idx):
    주소지=not_match_final.loc[i,'주소지 (조합, 법인 소재지)']
    농경체=not_match_final.loc[i,'농업경영체']
    농지면적=not_match_final.loc[i,'농지면적(실제경작)']
    합계=not_match_final.loc[i,'합계']
    비고=not_match_final.loc[i,'비고']
    시도=not_match_final.loc[i,'지자체명_시도']
    시군구=not_match_final.loc[i,'지자체명_시군구']
    구분=not_match_final.loc[i,'구분']
    작물종류=not_match_final.loc[i,'작물 종류_x']
    for j in not_match1_idx:
        if not_match1.loc[j,'작물 종류_x']==작물종류 and not_match1.loc[j,'구분']==구분 and not_match1.loc[j,'지자체명_시군구']==시군구 and not_match1.loc[j,'지자체명_시도']==시도 and not_match1.loc[j,'비고']==비고 and not_match1.loc[j,'주소지 (조합, 법인 소재지)']==주소지 and not_match1.loc[j,'농업경영체']==농경체 and not_match1.loc[j,'농지면적(실제경작)']==농지면적 and not_match1.loc[j,'합계']==합계:
            실제_작물종류_list.append(j)
            break
        else:
            pass

100%|██████████████████████████████████████████████████████████████████████████████| 2492/2492 [03:43<00:00, 11.15it/s]


In [16]:
not_match_1=not_match1.loc[실제_작물종류_list]
not_match_1.drop(columns=['작물 종류_x'],inplace=True) #원본 작물 종류 삭제
new_column={'작물 종류_y':'작물 종류'}
not_match_1.rename(columns=new_column,inplace=True)
cols=not_match_1.columns.tolist()
cols[7], cols[8] =cols[8], cols[7]
not_match_1=not_match_1[cols]

In [17]:
final=pd.concat([match_data,not_match_1],ignore_index=True)

In [18]:
final

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작)
0,2022,강원도,춘천시,농가,강원도 춘천시 신북읍 신샘밭로 395,1000000327,3,① 시설원예·특작,3,0,3951.0
1,2022,강원도,춘천시,농가,강원도 춘천시 신북읍 신북로 30-26,1000000332,4,① 시설원예·특작,4,0,3663.0
2,2023,강원도,춘천시,농가,춘천시 신북읍 신북로 30-26,1000000332,2,① 시설원예·특작,2,0,3663.0
3,2022,강원도,춘천시,농가,강원도 춘천시 신북읍 신북로 30-17,1000000336,3,① 시설원예·특작,3,0,7302.0
4,2023,강원도,춘천시,농가,춘천시 신북읍 신북로 30-17,1000000336,6,① 시설원예·특작,6,0,7302.0
...,...,...,...,...,...,...,...,...,...,...,...
12612,2023,충청북도,음성군,농가,음성군 소이면 장금로 18-39,3000461549,7,① 시설원예·특작,7,0,5667.8
12613,2023,경상북도,봉화군,농가,법전면 풍정길 270,3000642967,1,⑧ 기타 식량작물,1,0,1800.0
12614,2023,충청남도,금산군,농가,"충남 금산군 금산읍 동산3길 27, 101-612(산마루)",3000851064,4,① 시설원예·특작,4,0,2360.0
12615,2023,전라남도,영암군,농가,전라남도 영암군 삼호읍 송죽정길 15-12,3000873387,2,"④ 인삼, 일반채소",2,0,2674.0


# 7 파생변수 생성

## 7.1 농지면적 ha 단위 변환 및 구분 컬럼 표준화

In [19]:
final['농지면적(실제경작)']=round(final['농지면적(실제경작)']/10000,2)
final['구분']=final['구분'].apply(lambda x: re.sub(' ', '', x))  # 특수문자

## 7.2 작물 종류별 소농, 중농, 대농, 법인 구분

In [20]:
cate_value_1=final.groupby('작물 종류')['농지면적(실제경작)'].quantile([0.33])
cate_value_1=pd.DataFrame(cate_value_1).reset_index()
cate_value_1=cate_value_1[['작물 종류','농지면적(실제경작)']]
cate_value_2=final.groupby('작물 종류')['농지면적(실제경작)'].quantile([0.66])
cate_value_2=pd.DataFrame(cate_value_2).reset_index()
cate_value_2=cate_value_2[['작물 종류','농지면적(실제경작)']]
cate_value_1.columns=['작물 종류','cutoff1']
cate_value_2.columns=['작물 종류','cutoff2']
cate_value=pd.merge(cate_value_1,cate_value_2,on='작물 종류')
final=pd.merge(final,cate_value,on='작물 종류',how='left')

In [21]:
for i in tqdm(final.index):
    if final.loc[i,'구분']=='농가':
        if final.loc[i,'농지면적(실제경작)'] <= final.loc[i,'cutoff1']:
            final.loc[i,'구분']='소농'
        elif final.loc[i,'cutoff1'] < final.loc[i,'농지면적(실제경작)'] <= final.loc[i,'cutoff2']:
            final.loc[i,'구분']='중농'
        elif final.loc[i,'cutoff2'] < final.loc[i,'농지면적(실제경작)']:
            final.loc[i,'구분']='대농'
    else:
        pass
    

100%|██████████████████████████████████████████████████████████████████████████| 12617/12617 [00:03<00:00, 3393.33it/s]


In [22]:
final.drop(columns=['cutoff1','cutoff2'],inplace=True) #불필요 컬럼 삭제

## 7.3 전년대비 농경체 증감률 컬럼 생성

In [23]:
Regist=pd.read_excel(os.listdir()[13])

In [24]:
Regist['시군']=np.where(Regist['시군']=='세종시','세종특별자치시',Regist['시군'])
Regist['시군']=np.where(Regist['시군'].str.contains(r'\s'), Regist['시군'].str.slice(stop=3), Regist['시군']) #첫 세글자만 추출
#group화 진행
Regist=Regist.groupby(['연도','시도','시군']).sum('경영체수(건)').reset_index()

In [25]:
#2020=> 2021년도로, 2021=>2022년도로, 2022=>2023년도로
Regist_2021=Regist.query('연도==2020')
Regist_2021['연도']=2021
Regist_2022=Regist.query('연도==2021')
Regist_2022['연도']=2022
Regist_2023=Regist.query('연도==2022')
Regist_2023['연도']=2023

In [26]:
#2022년 증감률
Regist_2122=pd.merge(Regist_2021,Regist_2022,on=['시도','시군'])
Regist_2122['증감']=round(((Regist_2122['경영체수(건)_y']-Regist_2122['경영체수(건)_x'])/Regist_2122['경영체수(건)_x']) * 100,1)+100
Regist_2122.drop(columns=['연도_x','경영체수(건)_x','경영체수(건)_y'],inplace=True) #불필요 컬럼 삭제
Regist_2122.columns=['지자체명_시도','지자체명_시군구','비고','전년대비농경체증감률']
#2023년 증감률
Regist_2223=pd.merge(Regist_2022,Regist_2023,on=['시도','시군'])
Regist_2223['증감']=round(((Regist_2223['경영체수(건)_y']-Regist_2223['경영체수(건)_x'])/Regist_2223['경영체수(건)_x']) * 100,1)+100
Regist_2223.drop(columns=['연도_x','경영체수(건)_x','경영체수(건)_y'],inplace=True) #불필요 컬럼 삭제
Regist_2223.columns=['지자체명_시도','지자체명_시군구','비고','전년대비농경체증감률']
#결합
Regist=pd.concat([Regist_2122,Regist_2223],ignore_index=True)
#최종데이터셋 결합
final=pd.merge(final,Regist,on=['비고','지자체명_시도','지자체명_시군구'],how='left')

In [27]:
final

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작),전년대비농경체증감률
0,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신샘밭로 395,1000000327,3,① 시설원예·특작,3,0,0.40,101.5
1,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신북로 30-26,1000000332,4,① 시설원예·특작,4,0,0.37,101.5
2,2023,강원도,춘천시,중농,춘천시 신북읍 신북로 30-26,1000000332,2,① 시설원예·특작,2,0,0.37,102.5
3,2022,강원도,춘천시,대농,강원도 춘천시 신북읍 신북로 30-17,1000000336,3,① 시설원예·특작,3,0,0.73,101.5
4,2023,강원도,춘천시,대농,춘천시 신북읍 신북로 30-17,1000000336,6,① 시설원예·특작,6,0,0.73,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...
12612,2023,충청북도,음성군,대농,음성군 소이면 장금로 18-39,3000461549,7,① 시설원예·특작,7,0,0.57,102.9
12613,2023,경상북도,봉화군,소농,법전면 풍정길 270,3000642967,1,⑧ 기타 식량작물,1,0,0.18,101.7
12614,2023,충청남도,금산군,소농,"충남 금산군 금산읍 동산3길 27, 101-612(산마루)",3000851064,4,① 시설원예·특작,4,0,0.24,101.9
12615,2023,전라남도,영암군,소농,전라남도 영암군 삼호읍 송죽정길 15-12,3000873387,2,"④ 인삼, 일반채소",2,0,0.27,102.6


# 8. 지역별 농업인 현황 65세 농업인구 비율 컬럼 생성

In [28]:
use_col_list=list(range(5,18))
use_col=[0,1]
use_col=use_col+use_col_list

In [29]:
worker_2022=pd.read_excel(os.listdir()[9],skiprows=1,usecols=use_col)
worker_2022['비고']=2022
worker_2023=pd.read_excel(os.listdir()[10],skiprows=1,usecols=use_col)
worker_2023['비고']=2023

In [30]:
worker_2022.columns=['지자체명_시도', '지자체명_시군구', '25세 미만', '25~29세', '30~34세', '35~39세','40~44세', '45~49세', '50~54세', '55~59세', '60~64세', '65~69세', '70~74세','75~79세', '80세 이상','비고']
worker_2023.columns=['지자체명_시도', '지자체명_시군구', '25세 미만', '25~29세', '30~34세', '35~39세','40~44세', '45~49세', '50~54세', '55~59세', '60~64세', '65~69세', '70~74세','75~79세', '80세 이상','비고']
worker_2022=worker_2022[worker_2022['지자체명_시군구']!='소계']
worker_2023=worker_2023[worker_2023['지자체명_시군구']!='소계']
worker_2022=worker_2022[worker_2022['지자체명_시도']!='총계']
worker_2023=worker_2023[worker_2023['지자체명_시도']!='총계']

In [31]:
worker=pd.concat([worker_2022,worker_2023],ignore_index=True)
worker['지자체명_시군구']=np.where(worker['지자체명_시군구']=='세종시','세종특별자치시',worker['지자체명_시군구'])
worker['지자체명_시군구']=np.where(worker['지자체명_시군구'].str.contains(r'\s'), worker['지자체명_시군구'].str.slice(stop=3), worker['지자체명_시군구']) #첫 세글자만 추출

In [32]:
young_list=['25세 미만', '25~29세', '30~34세', '35~39세','40~44세', '45~49세', '50~54세', '55~59세', '60~64세','65~69세', '70~74세','75~79세', '80세 이상']
old_list=['65~69세', '70~74세','75~79세', '80세 이상']
young_sum=[]
old_sum=[]
for i in tqdm(worker.index):
    young_sum.append(worker.loc[i,young_list].sum())
    old_sum.append(worker.loc[i,old_list].sum())
worker['청년인구수']=young_sum
worker['노년인구수']=old_sum

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1378.29it/s]


In [33]:
worker=worker[['지자체명_시도', '지자체명_시군구','청년인구수','노년인구수','비고']]
worker=worker.groupby(['지자체명_시도','지자체명_시군구','비고']).sum(['청년인구수','노년인구수']).reset_index()
worker['고령농경체비율']=round(worker['노년인구수']/worker['청년인구수']*100,2)
worker=worker[['지자체명_시도','지자체명_시군구','비고','고령농경체비율']]

In [34]:
final=pd.merge(final,worker,on=['비고','지자체명_시도','지자체명_시군구'],how='left')
final

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작),전년대비농경체증감률,고령농경체비율
0,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신샘밭로 395,1000000327,3,① 시설원예·특작,3,0,0.40,101.5,49.39
1,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신북로 30-26,1000000332,4,① 시설원예·특작,4,0,0.37,101.5,49.39
2,2023,강원도,춘천시,중농,춘천시 신북읍 신북로 30-26,1000000332,2,① 시설원예·특작,2,0,0.37,102.5,51.68
3,2022,강원도,춘천시,대농,강원도 춘천시 신북읍 신북로 30-17,1000000336,3,① 시설원예·특작,3,0,0.73,101.5,49.39
4,2023,강원도,춘천시,대농,춘천시 신북읍 신북로 30-17,1000000336,6,① 시설원예·특작,6,0,0.73,102.5,51.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12612,2023,충청북도,음성군,대농,음성군 소이면 장금로 18-39,3000461549,7,① 시설원예·특작,7,0,0.57,102.9,53.35
12613,2023,경상북도,봉화군,소농,법전면 풍정길 270,3000642967,1,⑧ 기타 식량작물,1,0,0.18,101.7,54.69
12614,2023,충청남도,금산군,소농,"충남 금산군 금산읍 동산3길 27, 101-612(산마루)",3000851064,4,① 시설원예·특작,4,0,0.24,101.9,55.65
12615,2023,전라남도,영암군,소농,전라남도 영암군 삼호읍 송죽정길 15-12,3000873387,2,"④ 인삼, 일반채소",2,0,0.27,102.6,55.69


# 9. 참여자 현황 관련 파생변수 생성

In [35]:
particip=pd.read_excel(os.listdir()[11])

In [36]:
particip['년도'].unique()

array([2022, 2023, 2021], dtype=int64)

In [37]:
particip['소재불명발생일자'].isnull().value_counts()

True     30360
False     1832
Name: 소재불명발생일자, dtype: int64

In [38]:
particip_1=particip[(particip['구분'].isin(['결혼이민자가족','MOU'])) & (particip['업종']=='농업')]
particip_1=particip_1.query('년도==2021 or 년도==2022')

In [39]:
particip_1=particip_1.groupby(['농업경영체','년도']).agg(입국인원=('농업경영체','count'),전년도이탈인원=('소재불명발생일자',lambda x:x.notna().sum())).reset_index()

In [40]:
particip_1['비고']=particip_1['년도']+1

In [41]:
particip_1['전년도활용여부']=np.where(particip_1['입국인원']!=0,'Y','N')
particip_1['전년도이탈인원']=np.where(particip_1['전년도이탈인원']!=0,particip_1['전년도이탈인원'],0)

In [42]:
particip_1['입국인원'].isnull().value_counts()
particip_1['전년도이탈인원'].isnull().value_counts()

False    3373
Name: 전년도이탈인원, dtype: int64

In [43]:
particip_1=particip_1[['농업경영체','비고','전년도이탈인원','전년도활용여부']]

In [44]:
particip_1

,농업경영체,비고,전년도이탈인원,전년도활용여부
0,1.000000e+09,2023,0,Y
1,1.000000e+09,2023,0,Y
2,1.000000e+09,2023,0,Y
3,1.000001e+09,2023,3,Y
4,1.000001e+09,2023,0,Y
...,...,...,...,...
3368,3.000431e+09,2023,0,Y
3369,3.000462e+09,2023,0,Y
3370,3.000584e+09,2023,0,Y
3371,3.000828e+09,2023,0,Y


In [45]:
final_1=pd.merge(final,particip_1,on=['비고','농업경영체'],how='left')

In [46]:
final_1

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작),전년대비농경체증감률,고령농경체비율,전년도이탈인원,전년도활용여부
0,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신샘밭로 395,1000000327,3,① 시설원예·특작,3,0,0.40,101.5,49.39,NaN,NaN
1,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신북로 30-26,1000000332,4,① 시설원예·특작,4,0,0.37,101.5,49.39,NaN,NaN
2,2023,강원도,춘천시,중농,춘천시 신북읍 신북로 30-26,1000000332,2,① 시설원예·특작,2,0,0.37,102.5,51.68,0.0,Y
3,2022,강원도,춘천시,대농,강원도 춘천시 신북읍 신북로 30-17,1000000336,3,① 시설원예·특작,3,0,0.73,101.5,49.39,NaN,NaN
4,2023,강원도,춘천시,대농,춘천시 신북읍 신북로 30-17,1000000336,6,① 시설원예·특작,6,0,0.73,102.5,51.68,0.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12612,2023,충청북도,음성군,대농,음성군 소이면 장금로 18-39,3000461549,7,① 시설원예·특작,7,0,0.57,102.9,53.35,0.0,Y
12613,2023,경상북도,봉화군,소농,법전면 풍정길 270,3000642967,1,⑧ 기타 식량작물,1,0,0.18,101.7,54.69,NaN,NaN
12614,2023,충청남도,금산군,소농,"충남 금산군 금산읍 동산3길 27, 101-612(산마루)",3000851064,4,① 시설원예·특작,4,0,0.24,101.9,55.65,NaN,NaN
12615,2023,전라남도,영암군,소농,전라남도 영암군 삼호읍 송죽정길 15-12,3000873387,2,"④ 인삼, 일반채소",2,0,0.27,102.6,55.69,0.0,Y


In [47]:
final_1.loc[final_1['전년도이탈인원'].isnull(),'전년도이탈인원']=0

In [48]:
final_1.loc[final_1['전년도활용여부'].isnull(),'전년도활용여부']='N'

In [49]:
final_1

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작),전년대비농경체증감률,고령농경체비율,전년도이탈인원,전년도활용여부
0,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신샘밭로 395,1000000327,3,① 시설원예·특작,3,0,0.40,101.5,49.39,0.0,N
1,2022,강원도,춘천시,중농,강원도 춘천시 신북읍 신북로 30-26,1000000332,4,① 시설원예·특작,4,0,0.37,101.5,49.39,0.0,N
2,2023,강원도,춘천시,중농,춘천시 신북읍 신북로 30-26,1000000332,2,① 시설원예·특작,2,0,0.37,102.5,51.68,0.0,Y
3,2022,강원도,춘천시,대농,강원도 춘천시 신북읍 신북로 30-17,1000000336,3,① 시설원예·특작,3,0,0.73,101.5,49.39,0.0,N
4,2023,강원도,춘천시,대농,춘천시 신북읍 신북로 30-17,1000000336,6,① 시설원예·특작,6,0,0.73,102.5,51.68,0.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12612,2023,충청북도,음성군,대농,음성군 소이면 장금로 18-39,3000461549,7,① 시설원예·특작,7,0,0.57,102.9,53.35,0.0,Y
12613,2023,경상북도,봉화군,소농,법전면 풍정길 270,3000642967,1,⑧ 기타 식량작물,1,0,0.18,101.7,54.69,0.0,N
12614,2023,충청남도,금산군,소농,"충남 금산군 금산읍 동산3길 27, 101-612(산마루)",3000851064,4,① 시설원예·특작,4,0,0.24,101.9,55.65,0.0,N
12615,2023,전라남도,영암군,소농,전라남도 영암군 삼호읍 송죽정길 15-12,3000873387,2,"④ 인삼, 일반채소",2,0,0.27,102.6,55.69,0.0,Y


In [50]:
final_1.drop(columns=['주소지 (조합, 법인 소재지)'],inplace=True)

In [51]:
row_dup=final_1.duplicated()

In [52]:
final_1=final_1[-row_dup]

In [53]:
final_1.reset_index()

,index,비고,지자체명_시도,지자체명_시군구,구분,농업경영체,합계,작물 종류,배정신청 인원,지자체추가배정인원,농지면적(실제경작),전년대비농경체증감률,고령농경체비율,전년도이탈인원,전년도활용여부
0,0,2022,강원도,춘천시,중농,1000000327,3,① 시설원예·특작,3,0,0.40,101.5,49.39,0.0,N
1,1,2022,강원도,춘천시,중농,1000000332,4,① 시설원예·특작,4,0,0.37,101.5,49.39,0.0,N
2,2,2023,강원도,춘천시,중농,1000000332,2,① 시설원예·특작,2,0,0.37,102.5,51.68,0.0,Y
3,3,2022,강원도,춘천시,대농,1000000336,3,① 시설원예·특작,3,0,0.73,101.5,49.39,0.0,N
4,4,2023,강원도,춘천시,대농,1000000336,6,① 시설원예·특작,6,0,0.73,102.5,51.68,0.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12538,12612,2023,충청북도,음성군,대농,3000461549,7,① 시설원예·특작,7,0,0.57,102.9,53.35,0.0,Y
12539,12613,2023,경상북도,봉화군,소농,3000642967,1,⑧ 기타 식량작물,1,0,0.18,101.7,54.69,0.0,N
12540,12614,2023,충청남도,금산군,소농,3000851064,4,① 시설원예·특작,4,0,0.24,101.9,55.65,0.0,N
12541,12615,2023,전라남도,영암군,소농,3000873387,2,"④ 인삼, 일반채소",2,0,0.27,102.6,55.69,0.0,Y


In [55]:
final_1.to_csv('최종분석데이터셋_엄태원.csv',encoding='EUC-KR',index=False)